# Lab 4.2 Use generative Language Models as text generator

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

Most of you have used ChatGPT from OpenAI and noticed that it can respond to your input in a very natural way. ChatGPT is build on top of GPT (Generative Pretrained Transformer) which is a model trained to generate text given a preceding input, so-called prompt (Brown et al 2020). It can do this repetitively up to a certain length, likewise generating short stories.

Models such as ChatGPT, although having good performance, are by far too large model to work with locally. Therefore in this notebook, we will use smaller open source models such as Llama and Qwen which are publicly available on huggingface.co.

### References

Brown, T. B., Mann, B., Ryder, N., Subbiah, M., Kaplan, J., Dhariwal, P., Neelakantan, A., Shyam, P., Sastry, G., Askell, A., et al. Language models are few-shot learners. arXiv preprint arXiv:2005.14165, 2020.

OpenAI, 2023. GPT-4 Technical Report. arXiv:2303.08774

Llama: Touvron, Hugo, Thibaut Lavril, Gautier Izacard, Xavier Martinet, Marie-Anne Lachaux, Timothée Lacroix, Baptiste Rozière et al. "Llama: Open and efficient foundation language models." arXiv preprint arXiv:2302.13971 (2023). [The Llama 3 Herd of Models](https://scontent-ams2-1.xx.fbcdn.net/v/t39.2365-6/452387774_1036916434819166_4173978747091533306_n.pdf?_nc_cat=104&ccb=1-7&_nc_sid=3c67a6&_nc_ohc=t6egZJ8QdI4Q7kNvgEka7Z4&_nc_ht=scontent-ams2-1.xx&oh=00_AYBPnjp-CQn7YnUQU_P-yJATmlaN6oRuEHZ1VrXshBoBwQ&oe=66A6EB8D)

Qwen: Bai, Jinze, Shuai Bai, Yunfei Chu, Zeyu Cui, Kai Dang, Xiaodong Deng, Yang Fan et al. "Qwen technical report." arXiv preprint arXiv:2309.16609 (2023).

Generative LLMs are trained to respond to prompts. The prompt is given as a list of **dict** strings defining the role of the **system** (the model) and the input given by the **user**:

In [1]:
prompt = [{"role": "system", "content": "Generate 10 alternative sentences by completing the next input text."},
          {"role": "user", "content": "Bach sat down at his organ and played"}]

The ```transformers``` module can also be used to create a pipeline to generate a text as a response to prompt:

```# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="Qwen/Qwen3-1.7B-Base")
messages = [
    {"role": "user", "content": "Who are you?"},
]

response = pipe(messages)
print(response)
```

However it takes a lot of memory and time to download models.

Instead of the pipeline modules, we will use another package ```ChatOllama``` from **langchain** that was also used for your chat before.

In [3]:
from langchain_ollama import ChatOllama

model="qwen3:1.7b"

chat = ChatOllama(
            model=model,
            temperature= 0.1,
        )

In [4]:
response = chat.invoke(prompt)

### We need to remove the <think></think> part from the output
end_of_think = response.content.find("</think>")
answer = response
if end_of_think>0:
    think = response.content[:end_of_think+8]
    answer = response.content[end_of_think+8:].replace("\n", "")

In [5]:
print("Thinking aloud", think)
print()
print("The final answer", answer)

Thinking aloud <think>
Okay, the user wants me to generate 10 alternative sentences by completing the sentence "Bach sat down at his organ and played..." Let me start by understanding the original sentence. It's about Johann Sebastian Bach playing his organ. The user probably wants different ways to describe the action, maybe varying the verbs or adding details.

First, I need to think about different verbs that can replace "played." Words like "sang," "composed," "rehearsed," "performed," "played," "marched," "swept," "danced," "wrote," "recited," "marched." Wait, "marched" might not fit here. Maybe "swept" or "danced" could work. Also, adding adjectives or different structures.

Next, think about the context. Bach is known for his organ music, so maybe mention the piece or the effect. For example, "Bach sat down at his organ and played a sonata." Or "Bach sat down at his organ and played a fugue." Also, maybe include the audience or the reaction. "Bach sat down at his organ and playe

When we define the client, we can not only adjust the prompt but also the temperature. Temperatures closer to 1.0 make the model become more creative, closer to 1.0 it will follow the instructions more in a strict way. So let's try again with a high temperature.

In [6]:
chat_creative = ChatOllama(
            model=model,
            temperature= 0.9,
        )

In [7]:
response = chat_creative.invoke(prompt)

### We need to remove the <think></think> part from the output
end_of_think = response.content.find("</think>")
answer = response
if end_of_think>0:
    think = response.content[:end_of_think+8]
    answer = response.content[end_of_think+8:].replace("\n", "")

print(answer)

1. Bach sat down at his organ and played a sonata.  2. Bach sat down at his organ and played a concerto.  3. Bach sat down at his organ and played a cantata.  4. Bach sat down at his organ and played a fugue.  5. Bach sat down at his organ and played a symphony.  6. Bach sat down at his organ and played a chorale.  7. Bach sat down at his organ and played a piece.  8. Bach sat down at his organ and played a melody.  9. Bach sat down at his organ and played a composition.  10. Bach sat down at his organ and played a movement.


You can clearly see that it got more expressive.

Let us set the temperature back to 0.1 and try another one:

In [8]:
prompt = [{"role": "system", "content": "Generate 10 alternative sentences by completing the next input text."},
          {"role": "user", "content": "Beethoven sat down at his organ and played"}]


response = chat.invoke(prompt)

### We need to remove the <think></think> part from the output
end_of_think = response.content.find("</think>")
answer = response
if end_of_think>0:
    think = response.content[:end_of_think+8]
    answer = response.content[end_of_think+8:].replace("\n", "")

print(answer)

1. Beethoven sat down at his organ and composed a symphony.  2. Beethoven sat down at his organ and played a sonata.  3. Beethoven sat down at his organ and performed a piece.  4. Beethoven sat down at his organ and played a nocturne.  5. Beethoven sat down at his organ and played a fugue.  6. Beethoven sat down at his organ and played a prelude.  7. Beethoven sat down at his organ and played a piece that would later become a classic.  8. Beethoven sat down at his organ and played a piece that marked a new era in classical music.  9. Beethoven sat down at his organ and played a composition that inspired a new generation.  10. Beethoven sat down at his organ and played a melody that would forever change the landscape of classical music.


Let us switch the language of the input, as Qwen is a multilingual model:

In [9]:
prompt = [{"role": "system", "content": "Generate 10 alternative sentences by completing the next input text in Dutch."},
          {"role": "user", "content": "Beethoven ging achter zijn orgel zitten en speelde"}]


response = chat.invoke(prompt)

### We need to remove the <think></think> part from the output
end_of_think = response.content.find("</think>")
answer = response
if end_of_think>0:
    think = response.content[:end_of_think+8]
    answer = response.content[end_of_think+8:].replace("\n", "")

print(answer)

1. Beethoven ging achter zijn orgel zitten en speelde een lied dat hij had opgesteld.  2. Beethoven ging achter zijn orgel zitten en speelde een symfonie dat hij had opgesteld.  3. Beethoven ging achter zijn orgel zitten en speelde een konzert dat hij had opgesteld.  4. Beethoven ging achter zijn orgel zitten en speelde een motief dat hij had opgesteld.  5. Beethoven ging achter zijn orgel zitten en speelde een klok dat hij had opgesteld.  6. Beethoven ging achter zijn orgel zitten en speelde een opname dat hij had opgesteld.  7. Beethoven ging achter zijn orgel zitten en speelde een kantoor dat hij had opgesteld.  8. Beethoven ging achter zijn orgel zitten en speelde een kantoor dat hij had opgesteld.  9. Beethoven ging achter zijn orgel zitten en speelde een kantoor dat hij had opgesteld.  10. Beethoven ging achter zijn orgel zitten en speelde een kantoor dat hij had opgesteld.


Instructions in English, completions as Dutch. You can also give the instructions in Dutch if you want or try other languages.

In [ ]:
prompt = [{"role": "system", "content": "Genereer 10 alternatieve zinnen door de volgende tekst af te maken."},
          {"role": "user", "content": "Beethoven ging achter zijn orgel zitten en speelde"}]

response = chat.invoke(prompt)

### We need to remove the <think></think> part from the output
end_of_think = response.content.find("</think>")
answer = response
if end_of_think>0:
    think = response.content[:end_of_think+8]
    answer = response.content[end_of_think+8:].replace("\n", "")

print(answer)

So Dutch instructions do make a big difference as the quality went down drastically.

## End of notebook